In [5]:
import pandas as pd
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from pmdarima import auto_arima
from sklearn.preprocessing import MinMaxScaler

In [7]:
data = pd.read_csv('data/raw_data.csv')

In [8]:
data['TOTAL_DELAY'] = data['ARR_DELAY_NEW']
data1 = data.drop(columns=['CARRIER_NAME','AVG_MONTHLY_PASS_AIRLINE','ARR_DELAY_NEW','OP_CARRIER_FL_NUM', 'FLT_ATTENDANTS_PER_PASS', 'GROUND_SERV_PER_PASS', 'PLANE_AGE'])
data1 = data1.dropna(subset=['TOTAL_DELAY'])

In [9]:
order = ['0001-0559', '0600-0659', '0700-0759', '0800-0859', '0900-0959', '1000-1059', '1100-1159', '1200-1259', '1300-1359', '1400-1459', '1500-1559', '1600-1659', '1700-1759', '1800-1859', '1900-1959', '2000-2059', '2100-2159', '2200-2259', '2300-2359']
data1['DEP_TIME_BLK'] = pd.Categorical(data1['DEP_TIME_BLK'], categories=order, ordered=True)

In [10]:
data1.sort_values(['DATE', 'DEP_TIME_BLK'], inplace=True)
data_cleaned = data1.reset_index(drop=True)
print(data_cleaned.head())

   MONTH  DAY_OF_MONTH  DAY_OF_WEEK  DEP_DEL15 DEP_TIME_BLK  DISTANCE_GROUP  \
0      1             1            2          0    0800-0859               4   
1      1             1            2          0    0800-0859               4   
2      1             1            2          0    0800-0859               4   
3      1             1            2          0    0800-0859               4   
4      1             1            2          0    0800-0859               3   

   SEGMENT_NUMBER  CONCURRENT_FLIGHTS  NUMBER_OF_SEATS  AIRPORT_FLIGHTS_MONTH  \
0               1                   8              129                   3689   
1               1                   8              129                   3689   
2               1                   9              129                   3598   
3               1                  20              129                   8569   
4               2                   4              129                   1745   

   ...  LONGITUDE  PREVIOUS_AIRPORT   

In [11]:
data_cleaned.to_csv('data/data_cleaned.csv', index=False)

In [12]:
data_feature = data_cleaned.drop(columns=['TOTAL_DELAY'])
print(data_feature.nunique())
non_numeric_columns = data_feature.select_dtypes(include=['object']).columns.tolist()
print(non_numeric_columns)

MONTH                              12
DAY_OF_MONTH                       31
DAY_OF_WEEK                         7
DEP_DEL15                           2
DEP_TIME_BLK                       19
DISTANCE_GROUP                      8
SEGMENT_NUMBER                      9
CONCURRENT_FLIGHTS                 71
NUMBER_OF_SEATS                     2
AIRPORT_FLIGHTS_MONTH             493
AIRLINE_FLIGHTS_MONTH              12
AIRLINE_AIRPORT_FLIGHTS_MONTH     174
AVG_MONTHLY_PASS_AIRPORT           51
DEPARTING_AIRPORT                  51
LATITUDE                           51
LONGITUDE                          51
PREVIOUS_AIRPORT                  120
DATE                              364
PRCP                              246
SNOW                               48
SNWD                               15
TMAX                              112
AWND                              118
FLIGHT_NUMBER                    1855
dtype: int64
['DEPARTING_AIRPORT', 'PREVIOUS_AIRPORT', 'DATE']


In [13]:
# Ordinal encode the DEP_TIME_BLK column
ordinal_encoder = OrdinalEncoder()
data_feature['DEP_TIME_BLK'] = ordinal_encoder.fit_transform(data_feature[['DEP_TIME_BLK']])

# Target encode the remaining two labels
binary_encoder = ce.BinaryEncoder(cols=non_numeric_columns[:-1])
data_encoded = binary_encoder.fit_transform(data_feature)

Calculate residuals: 

In [14]:
model = auto_arima(data_cleaned['TOTAL_DELAY'], seasonal=True, trace=True,
                   error_action='ignore', suppress_warnings=True)
print(model.order)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=458436.475, Time=12.68 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=486723.249, Time=0.37 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=474256.245, Time=0.81 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=458446.287, Time=4.71 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=486721.249, Time=0.19 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=458449.271, Time=7.88 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=17.76 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=15.17 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=458449.325, Time=28.89 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=458438.193, Time=12.06 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=458436.558, Time=9.65 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=25.09 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=458436.847, Time=25.59 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=inf, Time=10.36 sec

Best model:  ARIMA(2,1,2

In [15]:
arima_predictions = model.predict_in_sample()
residuals = data_cleaned['TOTAL_DELAY'] - arima_predictions
print(residuals)

0         0.000002
1         1.000002
2        -0.505216
3        -0.329428
4        -0.238046
           ...    
42523   -15.980661
42524   -15.375962
42525   -14.901636
42526   -14.998255
42527    80.429708
Length: 42528, dtype: float64


Do not continas flight number as feature, because it is not a numerical value.

In [16]:
data_encoded['RESIDUALS'] = residuals
data_feature_encoded = data_encoded.drop(columns=['FLIGHT_NUMBER', 'DATE'])

data_target = data_cleaned['TOTAL_DELAY']
correlations = data_feature_encoded.corrwith(data_target)
print(correlations)

MONTH                            0.005440
DAY_OF_MONTH                    -0.005215
DAY_OF_WEEK                      0.016262
DEP_DEL15                        0.507516
DEP_TIME_BLK                     0.089687
DISTANCE_GROUP                   0.002379
SEGMENT_NUMBER                   0.068513
CONCURRENT_FLIGHTS              -0.057211
NUMBER_OF_SEATS                  0.004010
AIRPORT_FLIGHTS_MONTH           -0.050420
AIRLINE_FLIGHTS_MONTH            0.013235
AIRLINE_AIRPORT_FLIGHTS_MONTH   -0.069500
AVG_MONTHLY_PASS_AIRPORT        -0.045253
DEPARTING_AIRPORT_0              0.009234
DEPARTING_AIRPORT_1              0.036050
DEPARTING_AIRPORT_2              0.005721
DEPARTING_AIRPORT_3             -0.024223
DEPARTING_AIRPORT_4             -0.031905
DEPARTING_AIRPORT_5              0.058747
LATITUDE                        -0.013255
LONGITUDE                        0.063778
PREVIOUS_AIRPORT_0               0.021528
PREVIOUS_AIRPORT_1               0.000359
PREVIOUS_AIRPORT_2               0

In [17]:
scaler = MinMaxScaler()
data_encoded_scaled = scaler.fit_transform(data_feature_encoded)
data_encoded_scaled = pd.DataFrame(data_encoded_scaled, columns=data_feature_encoded.columns)

In [19]:
data_encoded_scaled.to_csv('data/data_encoded_scaled.csv', index=False)

data_cleaned: the dataset containing all information and departure delays, including flight number.

data_feature: same as above without delay

data_feature_encoded: performed encoding, discarding 'FLIGHT_NUMBER', 'DATE' these features that we will directly apply to make sequence

data_encoded_scaled: completely processed and standardized features

data_target: departure delay 